In [ ]:
!git clone https://github.com/airtlab/ConvLSTM4AD

#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!pip install simpleitk
#!pip show tensorflow
#!pip install --upgrade tensorflow

import SimpleITK as sitk
import os
import nibabel as nib
import numpy as np
import tensorflow as tf

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [ ]:
def resample_img(itk_image, out_spacing=[1, 1, 1], is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()
    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())
    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)
    return resample.Execute(itk_image)

def preprocess(img, out_shape=None):
    if out_shape is not None:
        img = resize(img, out_shape, mode='constant')
    
    mean = img.mean() 
    std = img.std() 
    return (img - mean) / std

In [ ]:
img_path = "/content/ConvLSTM4AD/Merged_dataset/Dataset"
img_height, img_width = 256, 256 
seq_len = 5

X = [] # an empty list ‘X’ is generated
Y = [] # an empty list ‘Y’ is generated

dir = img_path # the content of the string ‘img_path’ is stored in a new string ‘dir’ to simplify its subsequent usage
classes = ["AD","CN"] 
classes_list = os.listdir(dir) # a list containing the names of the elements inside ‘dir’ is generated
classes_list.sort() # the list is sorted to have AD as first element and CN as second one

full_path_AD = os.path.join(dir, classes_list[0]) # the path of the AD directory is obtained joining ‘dir’ with the first element of ‘classes_list’
samples_AD = os.listdir(full_path_AD) # a list containing the names of the elements inside the AD directory is generated 
samples_AD.sort() # the sample names are sorted alphabetically 
for a in samples_AD: 
  b = os.path.join(full_path_AD, a) # the file path is stored joining ‘full_path_AD’ with the current sample name ‘a’ 
  my_img = nib.load(b) 
  nii_data = my_img.get_fdata() 
  my_img.uncache()
  del my_img
  temp = [] # a new empty list ‘temp’ is generated
  for i in range(0,5): 
    temp.append(preprocess(nii_data[:,:,i]))
  X.append(temp) # once all slices have been appended, ‘temp’ is appended to the ‘X’ list
  y = [0]*len(classes_list) # a 1x2 array ‘y’ filled with zeros is generated
  y[1] = 1 # 1 is assigned to the second column of the array ([0,1] is the AD label)
  Y.append(y) # the ‘y’ array is appended to the ‘Y’ list

full_path_CN = os.path.join(dir, classes_list[1]) # the path of the CN directory is obtained joining ‘dir’ with the second element of ‘classes_list’
samples_CN = os.listdir(full_path_CN) 
samples_CN.sort()
for a in samples_CN: 
  b = os.path.join(full_path_CN, a)
  my_img = nib.load(b) 
  nii_data = my_img.get_fdata() 
  my_img.uncache()
  del my_img
  temp = []
  for i in range(0,5): 
    temp.append(preprocess(nii_data[:,:,i]))
  X.append(temp)
  y = [0]*len(classes_list)
  y[0] = 1 # 1 is assigned to the first column of the array ([1,0] is the CN label)
  Y.append(y)  

In [ ]:
X = np.asarray(X) # ‘X’ is turned into a numpy array
X = X[...,np.newaxis] # a new axis is added to ‘X’ to match the shape requested by the training function

Y = np.asarray(Y) # the same for ‘Y’

#print(X.shape)
#print(Y.shape)

In [ ]:
seed = 0 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, 
                                                    shuffle=True, random_state=seed) # split in train (train+val) and test: 80% of the total dataset for train+val and 20% for test 
                                                                                     # ‘shuffle’ shuffles all data before splitting
                                                                                     # ‘random_state’ controls the shuffling: each integer is related to a specific shuffle

del X
del Y

In [ ]:
model = models.Sequential()
model.add(layers.ConvLSTM2D(filters=32, 
                            kernel_size=(3,3), 
                            return_sequences=False, 
                            data_format="channels_last", 
                            input_shape=(seq_len,img_height,img_width,1)))
model.add(layers.Dropout(0.5)) 
model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.5)) 
model.add(layers.Dense(2, activation="softmax"))

#model.summary()

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001) 
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

earlystop = tf.keras.callbacks.EarlyStopping(patience=7)
callbacks = [earlystop]
history = model.fit(x=X_train, y=Y_train, epochs=40, batch_size=4, shuffle=True, validation_split=0.2, callbacks=callbacks)